# 06 - Model Risk: Stochastic Volatility and Smile

Generate a smile from a Heston-lite world and compare hedging errors against the Black-Scholes world.

In [ ]:
from pathlib import Path
import sys

ROOT = Path.cwd()
if not (ROOT / 'src').exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

import numpy as np
import pandas as pd

from src.config import config_dict
from src.stoch_vol import simulate_heston_lite_paths, sv_option_prices_mc, implied_vol_smile_from_sv_prices
from src.hedging import simulate_delta_hedge_on_paths, simulate_delta_hedge_gbm
from src.plotting import plot_smile

cfg = config_dict(fast_mode=True)


In [ ]:
strike_grid = [80, 90, 100, 110, 120]
maturities = [0.5, 1.0]
price_table = sv_option_prices_mc(
    strike_grid=strike_grid,
    maturities=maturities,
    S0=cfg['S0'],
    r=cfg['R'],
    q=cfg['Q'],
    V0=cfg['V0'],
    kappa=cfg['KAPPA'],
    theta=cfg['THETA'],
    xi=cfg['XI'],
    rho=cfg['RHO'],
    n_paths=40_000,
    n_steps_per_year=252,
    option_type='call',
    seed=cfg['SEED'],
)
smile = implied_vol_smile_from_sv_prices(price_table, S=cfg['S0'], r=cfg['R'], q=cfg['Q'])
smile.to_csv(ROOT / 'results' / 'tables' / 'sv_smile.csv', index=False)
plot_smile(smile, ROOT / 'results' / 'figures' / 'sv_smile.png')
smile.head()


In [ ]:
atm_row = smile[(smile['maturity'] == 1.0) & (smile['strike'] == cfg['K'])]
if atm_row.empty:
    sigma_hedge = float(smile['implied_vol'].dropna().median())
else:
    sigma_hedge = float(atm_row['implied_vol'].iloc[0])

sv_paths = simulate_heston_lite_paths(
    S0=cfg['S0'], r=cfg['R'], q=cfg['Q'], V0=cfg['V0'], kappa=cfg['KAPPA'], theta=cfg['THETA'],
    xi=cfg['XI'], rho=cfg['RHO'], T=1.0, n_paths=6_000, n_steps=252, seed=cfg['SEED']
)
sv_hedge = simulate_delta_hedge_on_paths(
    paths=sv_paths, K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma_model=sigma_hedge,
    T=1.0, rebalance_every_k_steps=5, option_type='call', tx_cost_per_dollar=0.0
)
bs_hedge = simulate_delta_hedge_gbm(
    S0=cfg['S0'], K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma=sigma_hedge, T=1.0,
    n_paths=6_000, n_steps=252, rebalance_every_k_steps=5, option_type='call',
    tx_cost_per_dollar=0.0, seed=cfg['SEED']
)

model_risk = pd.DataFrame([
    {'world': 'BS', 'mean_error': bs_hedge['mean_error'], 'std_error': bs_hedge['std_error'], 'q05': bs_hedge['q05'], 'q95': bs_hedge['q95']},
    {'world': 'SV_misspecified_delta', 'mean_error': sv_hedge['mean_error'], 'std_error': sv_hedge['std_error'], 'q05': sv_hedge['q05'], 'q95': sv_hedge['q95']},
])
model_risk.to_csv(ROOT / 'results' / 'tables' / 'model_risk_hedge_comparison.csv', index=False)
model_risk
